In [ ]:
import bagpy
from bagpy import bagreader
import os
import pandas as pd
from dateutil.parser import parse as parsedate

def biases_over_days(rat, path, number_of_days):
    if isinstance(rat, str):
        rat = int(rat)

    anim_folder = os.path.join(path, 'NC4%04d' % rat)
    training_folders = get_latest_folders(anim_folder, number_of_days)
    statistics = {}
    for folder in training_folders:
        folder_name = os.path.basename(folder)
        biases = extract_biases (rat, path, folder_name)
        statistics[folder_name] = list(biases)
    return statistics
        
def get_latest_folders(directory, n=7):
   
    subdirs = [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    
    subdirs.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    
    latest_subdirs = subdirs[:n]
    
    return latest_subdirs
    
def extract_biases(rat, path, date):
    if isinstance(rat, str):
        rat = int(rat)

    anim_folder = os.path.join(path, 'NC4%04d' % rat)

    if '-' in date:  # e.g. date = '24-Jul-24'
        date = parsedate(date).strftime('%y%m%d')

    date_folder = os.path.join(anim_folder, date)
    
    bag_file = None  

    for file in os.listdir(date_folder):
        if file.endswith('.bag'):
            bag_file = os.path.join(date_folder, file)  
            break 

    if bag_file is None:
        raise FileNotFoundError("No .bag file found in the specified folder.")
    
    data = bagreader(bag_file)

    rosout_topic_csv = data.message_by_topic('/rosout')

    rosout_topic = pd.read_csv(rosout_topic_csv)

    csv_file_path = os.path.join(date_folder, 'rosout.csv')

    rosout_topic.to_csv(csv_file_path, index=False)

    totall_number_of_trials = len(rosout_topic[rosout_topic['msg'].str.startswith('Current')]['msg'])

    wrong_choices = len(rosout_topic[rosout_topic['msg'] == 'ERROR']['msg'])

    correct_choices = len(rosout_topic[rosout_topic['msg'] == 'SUCCESS']['msg'])

    left_choices = len(rosout_topic[rosout_topic['msg'].str.startswith("Left")]['msg'])

    right_choices = len(rosout_topic[rosout_topic['msg'].str.startswith("Right")]['msg'])

    chamber_one_selected = len(rosout_topic[rosout_topic['msg'].str.endswith('is 1')]['msg'])

    chamber_three_selected = len(rosout_topic[rosout_topic['msg'].str.endswith('is 3')]['msg'])

    chamber_five_selected = len(rosout_topic[rosout_topic['msg'].str.endswith('is 5')]['msg'])

    chamber_seven_selected = len(rosout_topic[rosout_topic['msg'].str.endswith('is 7')]['msg'])

    return totall_number_of_trials, wrong_choices, correct_choices, left_choices, right_choices, chamber_one_selected, chamber_three_selected, chamber_five_selected, chamber_seven_selected

